In [1]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 3
import os
import time
from itertools import islice, permutations, product
from collections import namedtuple
from importlib import reload
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import IPython.display as display

from tensorflow.contrib.layers import fully_connected, flatten

from magnolia.features.mixer import FeatureMixer
from magnolia.features.wav_iterator import batcher
from magnolia.features.hdf5_iterator import SplitsIterator, Hdf5Iterator
import magnolia.features.spectral_features as sf
from magnolia.utils.postprocessing import reconstruct
reload(sf)
from magnolia.utils.tf_utils import scope_decorator as scope
from magnolia.dnnseparate.pit import PITModel

%matplotlib inline

In [ ]:
num_srcs = 2
num_steps = 51
num_freq_bins = 257
batch_size = 256

checkpoint_path = "/local_data/pcallier/pit_checkpoint"

def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def scale_spectrogram(spectrogram):
    mag_spec = np.abs(spectrogram)
    phases = np.unwrap(np.angle(spectrogram))
    
    mag_spec = np.sqrt(mag_spec)
    M = mag_spec.max()
    m = mag_spec.min()
    
    return (mag_spec - m)/(M - m), phases

In [ ]:
tf.reset_default_graph()

model_name = "pit_dnn_{}spkr".format(num_srcs)
model = PITModel('pit-s-dnn', num_srcs, num_steps, num_freq_bins, learning_rate=5e-4)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_session_start = time.time()

losses = []
val_losses = []
weight_changes = []
train_times  = []
MbStats = namedtuple('MbStats',['i', 'loss', 'time_delta', 'timestamp', 'batch_size'])
train_stats = []

In [ ]:
librispeech_dev = "/local_data/teams/magnolia/librispeech/processed_dev-clean.h5"
librispeech_train = "/local_data/teams/magnolia/librispeech/processed_train-clean-100.h5"
librispeech_test = "/local_data/teams/magnolia/librispeech/processed_test_clean.h5"

feature_iterators = [SplitsIterator([0.8,0.1,0.1], librispeech_train, 
                     shape=(num_steps, None), seed=i) for i in range(num_srcs)]
mixed_features = FeatureMixer(feature_iterators, shape=(num_steps, None))
data_batches = batcher(mixed_features, batch_size)

feature_iterators_val_inset = [SplitsIterator([0.8,0.1,0.1], librispeech_train, 
                     shape=(num_steps, None), seed=i) for i in range(num_srcs)]
for i in range(num_srcs):
    feature_iterators_val_inset[i].set_split(1)
mixed_features_val_inset = FeatureMixer(feature_iterators_val_inset, shape=(num_steps, None))
data_batches_val_inset = batcher(mixed_features_val_inset, batch_size)

feature_iterators_test_inset = [SplitsIterator([0.8,0.1,0.1], librispeech_train, 
                     shape=(num_steps, None), seed=i) for i in range(num_srcs)]
for i in range(num_srcs):
    feature_iterators_test_inset[i].set_split(2)
mixed_features_test_inset = FeatureMixer(feature_iterators_test_inset, shape=(num_steps, None))
data_batches_test_inset = batcher(mixed_features_test_inset, batch_size)

feature_iterators_val_outset = [Hdf5Iterator(librispeech_dev, 
                     shape=(num_steps, None), seed=i) for i in range(num_srcs)]
mixed_features_val_outset = FeatureMixer(feature_iterators_val_outset, shape=(num_steps, None))
data_batches_val_outset = batcher(mixed_features_val_outset, batch_size)

feature_iterators_test_outset = [Hdf5Iterator(librispeech_test, 
                     shape=(num_steps, None), seed=i) for i in range(num_srcs)]
mixed_features_test_outset = FeatureMixer(feature_iterators_test_outset, shape=(num_steps, None))
data_batches_test_outset = batcher(mixed_features_test_outset, batch_size)

## 3 speakers

In [ ]:
%pdb off
prev_first_layer_weights = None
prev_weights = None
saver = tf.train.Saver()
nbatches = 100000 if num_srcs == 3 else 0
for i, batch in enumerate(islice(data_batches, nbatches)):
    
    # Get training data
    batch_features, batch_ref1, batch_ref2, batch_ref3 = batch
    batch_ref1_norm, batch_ref1_norm_phase = scale_spectrogram(batch_ref1) 
    batch_ref2_norm, batch_ref2_norm_phase = scale_spectrogram(batch_ref2)
    batch_ref3_norm, batch_ref3_norm_phase = scale_spectrogram(batch_ref3)
    batch_features_norm, batch_features_norm_phase = scale_spectrogram(batch_features)
    
    # Do optimization
    data = { 
        model.X_in: batch_features_norm,
        model.y_in: np.stack((batch_ref1_norm, batch_ref2_norm, batch_ref3_norm), axis=1)
        }
    sess.run(model.optimize, feed_dict=data)
    loss = sess.run(model.loss, feed_dict=data)
    losses.append(loss)
    
    # get validation statistics
    val_batch = next(val_batches)
    (val_feat_norm, _), (val_ref1_norm, _), (val_ref2_norm, _), (val_ref3_norm, _) = \
        [scale_spectrogram(x) for x in val_batch]
    val_data = { model.X_in: val_feat_norm, 
                model.y_in: np.stack((val_ref1_norm, val_ref2_norm, val_ref3_norm), axis=1)}
    val_loss = sess.run(model.loss, feed_dict=val_data)
    val_losses.append(val_loss)
    
    # Collect predictions, gradient statistics, masks
    prediction = sess.run(model.predict, data)
    all_weights = sess.run(tf.concat([tf.reshape(var, [-1]) for var in tf.trainable_variables()], axis=0), data)
    if prev_weights is not None:
        weight_changes.append(np.linalg.norm(all_weights-prev_weights))
    prev_weights = all_weights
    if model.mask is not None:
        masks = sess.run(model.mask, data)
    else:
        masks = None
    
    # Checkpointing
    if len(losses) > 0 and len(losses) % 1000 == 0:
#     if True:
        saver.save(sess, os.path.join(checkpoint_path, "{model}_{batch}_{stamp}.ckpt".format(
            model=model_name,
            batch=len(losses),
            stamp=hex(hash(time.time()))[2:8]
        )))
    
    # Plotting
    if len(losses) > 0 and len(losses) % 10 == 0:
#     if True:
        orig = reconstruct(batch_features[0], batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        recon_a = reconstruct(prediction[0,0]**2, batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        recon_b = reconstruct(prediction[0,1]**2, batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        recon_c = reconstruct(prediction[0,2]**2, batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        orig_a = reconstruct(batch_ref1[0], batch_ref1[0], fs=10000, window_size=None, step_size=0.0256)
        orig_b = reconstruct(batch_ref2[0], batch_ref2[0], fs=10000, window_size=None, step_size=0.0256)
        orig_c = reconstruct(batch_ref3[0], batch_ref3[0], fs=10000, window_size=None, step_size=0.0256)
        
        display.clear_output(wait=True)
        plt.figure(figsize=(8,9))
        plt.subplot(3,4,1); plt.imshow(np.sqrt(prediction[0,0].T), cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,2); plt.imshow(np.sqrt(prediction[0,1].T), cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,3); plt.imshow(np.sqrt(prediction[0,2].T), cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,4); plt.imshow(batch_features_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        if masks is not None:
            plt.subplot(3,4,5); plt.imshow(masks[0,0].T, cmap='bone', origin='lower', aspect=0.25, vmin=0, vmax=1)
            plt.subplot(3,4,6); plt.imshow(masks[1,0].T, cmap='bone', origin='lower', aspect=0.25, vmin=0, vmax=1)
            plt.subplot(3,4,7); plt.imshow(masks[2,0].T, cmap='bone', origin='lower', aspect=0.25, vmin=0, vmax=1)
            plt.subplot(3,4,8); plt.imshow(masks[2,1].T, cmap='bone', origin='lower', aspect=0.25, vmin=0, vmax=1)
            plt.title("Next example")
            
        plt.subplot(3,4,9); plt.imshow(batch_ref1_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,10); plt.imshow(batch_ref2_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,11); plt.imshow(batch_ref3_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,12); plt.imshow(batch_features_norm[1].T, cmap='bone', origin='lower', aspect=0.25)
        
        
        
        smooth_factor = len(losses) // 10
        if smooth_factor <= 5:
            avg_losses = losses
            avg_val = val_losses
            avg_chg = weight_changes
        else:
            avg_losses = moving_average(losses, smooth_factor)
            avg_val = moving_average(losses)
            avg_chg = moving_average(weight_changes, smooth_factor)
        plt.figure(figsize=(8,5))
        plt.plot(avg_losses)
        plt.plot(avg_val, color='red')
        plt.title("Losses")
        
        plt.figure(figsize=(8,5))
        plt.plot(avg_chg)
        plt.title("Norm of weight changes from previous minibatch")
        
        plt.figure()
        plt.imshow(sess.run(model.logits, data), cmap='afmhot', aspect=3)
        plt.colorbar()
        plt.show()
        print("Mix")
        display.display(display.Audio(orig, rate=10000))
        print("Recon A")
        display.display(display.Audio(recon_a, rate=10000))
        print("Recon B")
        display.display(display.Audio(recon_b, rate=10000))
        print("Recon C")
        display.display(display.Audio(recon_c, rate=10000))
        print("Originals")
        display.display(display.Audio(orig_a, rate=10000))
        display.display(display.Audio(orig_b, rate=10000))
        display.display(display.Audio(orig_c, rate=10000))

## 2 speakers

In [ ]:
%pdb off
prev_first_layer_weights = None
prev_weights = None
saver = tf.train.Saver()
nbatches = 100000 if num_srcs == 2 else 0
for i, batch in enumerate(islice(data_batches, nbatches)):
    
    # Get training data
    batch_features, batch_ref1, batch_ref2 = batch
    batch_ref1_norm, batch_ref1_norm_phase = scale_spectrogram(batch_ref1) 
    batch_ref2_norm, batch_ref2_norm_phase = scale_spectrogram(batch_ref2)
    batch_features_norm, batch_features_norm_phase = scale_spectrogram(batch_features)
    
    # Do optimization
    data = { 
        model.X_in: batch_features_norm,
        model.y_in: np.stack((batch_ref1_norm, batch_ref2_norm), axis=1)
        }
    train_start = time.time()
    sess.run(model.optimize, feed_dict=data)
    train_end = time.time()
    loss = sess.run(model.loss, feed_dict=data)
    losses.append(loss)
    
    minibatch_stats = MbStats(len(losses), loss, train_end - train_start, train_start, batch_size)
    train_stats.append(minibatch_stats)
    
    # get validation statistics (in-set)
    val_batch = next(data_batches_val_inset)
    (val_feat_norm, _), (val_ref1_norm, _), (val_ref2_norm, _) = \
        [scale_spectrogram(x) for x in val_batch]
    val_data = { model.X_in: val_feat_norm, 
                model.y_in: np.stack((val_ref1_norm, val_ref2_norm), axis=1)}
    val_loss = sess.run(model.loss, feed_dict=val_data)
    val_losses.append(val_loss)
    
    # Collect predictions, gradient statistics, masks
    prediction = sess.run(model.predict, data)
    all_weights = sess.run(tf.concat([tf.reshape(var, [-1]) for var in tf.trainable_variables()], axis=0), data)
    if prev_weights is not None:
        weight_changes.append(np.linalg.norm(all_weights-prev_weights))
    prev_weights = all_weights
    if model.mask is not None:
        masks = sess.run(model.mask, data)
    else:
        masks = None
        
    
    
    # Checkpointing
    if len(losses) > 0 and len(losses) % 1000 == 0:
#     if True:
        saver.save(sess, os.path.join(checkpoint_path, "{model}_{batch}_{stamp}.ckpt".format(
            model=model_name,
            batch=len(losses),
            stamp=hex(hash(time.time()))[2:8]
        )))

    
    # Plotting
    if len(losses) > 0 and len(losses) % 10 == 0:
#     if True:
        with open(os.path.join(checkpoint_path, "{model}_stats.txt".format(
            model=model_name
        )), "a") as f:
            for line in train_stats:
                f.write('\t'.join([str(x) for x in line]))
                f.write('\n')
            train_stats = []
            
        orig = reconstruct(batch_features[0], batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        recon_a = reconstruct(prediction[0,0]**2, batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        recon_b = reconstruct(prediction[0,1]**2, batch_features[0], fs=10000, window_size=None, step_size=0.0256)
        orig_a = reconstruct(batch_ref1[0], batch_ref1[0], fs=10000, window_size=None, step_size=0.0256)
        orig_b = reconstruct(batch_ref2[0], batch_ref2[0], fs=10000, window_size=None, step_size=0.0256)
        
        display.clear_output(wait=True)
        plt.figure(figsize=(8,9))
        plt.subplot(3,4,1); plt.imshow(np.sqrt(prediction[0,0].T), cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,2); plt.imshow(np.sqrt(prediction[0,1].T), cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,4); plt.imshow(batch_features_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        if masks is not None:
            plt.subplot(3,4,5); plt.imshow(masks[0,0].T, cmap='bone', origin='lower', aspect=0.25, vmin=0, vmax=1)
            plt.subplot(3,4,6); plt.imshow(masks[1,0].T, cmap='bone', origin='lower', aspect=0.25, vmin=0, vmax=1)
            
        plt.subplot(3,4,9); plt.imshow(batch_ref1_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        plt.subplot(3,4,10); plt.imshow(batch_ref2_norm[0].T, cmap='bone', origin='lower', aspect=0.25)
        
        
        smooth_factor = len(losses) // 10
        if smooth_factor <= 5:
            avg_losses = losses
            avg_val = val_losses
            avg_chg = weight_changes
        else:
            avg_losses = moving_average(losses, smooth_factor)
            avg_val = moving_average(val_losses, smooth_factor)
            avg_chg = moving_average(weight_changes, smooth_factor)
        plt.figure(figsize=(8,5))
        plt.plot(avg_losses)
        plt.plot(avg_val, color='red')
        plt.title("Losses")
        
        plt.figure(figsize=(8,5))
        plt.plot(avg_chg)
        plt.title("Norm of weight changes from previous minibatch")
        
        plt.figure()
        plt.imshow(sess.run(model.logits, data), cmap='afmhot', aspect=3)
        plt.colorbar()
        plt.show()
        print("Mix")
        display.display(display.Audio(orig, rate=10000))
        print("Recon A")
        display.display(display.Audio(recon_a, rate=10000))
        print("Recon B")
        display.display(display.Audio(recon_b, rate=10000))

        print("Originals")
        display.display(display.Audio(orig_a, rate=10000))
        display.display(display.Audio(orig_b, rate=10000))


In [ ]:
[str( x )for x in minibatch_stats]